# **1. Exploratory Data Analysis**

In [1]:
#Import data training
import pandas as pd

df_train = pd.read_csv('train.csv')

In [3]:
#nampilin 5 data teratas
df_train.head()

,review_id,review,rating
0,0,Ga disappointed neat products .. Meletot Hilsn...,1
1,1,"Rdtanya replace broken glass, broken chargernya",1
2,2,Nyesel bngt dsni shopping antecedent photo mes...,1
3,3,Sent a light blue suit goods ga want a refund,1
4,4,Pendants came with dents and scratches on its ...,1


In [4]:
#nampilin 5 data terbawah
df_train.tail()

,review_id,review,rating
146806,146806,Excellent product quality delivery speed is ve...,5
146807,146807,thanks gan,5
146808,146808,Awesome awesome quality merchandise value CP ...,5
146809,146809,Nice Packing boxes made effective price .........,5
146810,146810,Excellent product quality excellent product p...,5


In [5]:
#menampilkan berapa banyak jumlah labels di dataset 
df_train['rating'].value_counts()

4    41865
5    41515
3    35941
1    14785
2    12705
Name: rating, dtype: int64

In [6]:
#Mengubah value dari label. contoh 5 menjadi 4, 4 menjadi 3, dst.
labels = []

for index, row in df_train.iterrows():
  if row['rating'] == 5:
    labels.append(4)
  elif row['rating'] == 4 :
    labels.append(3)
  elif row['rating'] == 3:
    labels.append(2)
  elif row['rating'] == 2:
    labels.append(1)
  else :
    labels.append(0)

df_train['labels'] = labels
#menghapus kolom 'rating' karena akan digantikan dengan kolom 'labels'
df_train = df_train.drop(columns = 'rating')
df_train

,review_id,review,labels
0,0,Ga disappointed neat products .. Meletot Hilsn...,0
1,1,"Rdtanya replace broken glass, broken chargernya",0
2,2,Nyesel bngt dsni shopping antecedent photo mes...,0
3,3,Sent a light blue suit goods ga want a refund,0
4,4,Pendants came with dents and scratches on its ...,0
...,...,...,...
146806,146806,Excellent product quality delivery speed is ve...,4
146807,146807,thanks gan,4
146808,146808,Awesome awesome quality merchandise value CP ...,4
146809,146809,Nice Packing boxes made effective price .........,4


In [7]:
#menampilkan berapa banyak jumlah labels di dataset 
df_train['labels'].value_counts()

3    41865
4    41515
2    35941
0    14785
1    12705
Name: labels, dtype: int64

In [8]:
#copy isi df_train ke df_train_preprocessed
df_train_preprocessed = df_train.copy()
#hapus kolom 'review_id' pada df_train_preprocessed karena kolom ini tidak digunakan untuk training model
df_train_preprocessed = df_train.drop(columns = 'review_id')
df_train_preprocessed

,review,labels
0,Ga disappointed neat products .. Meletot Hilsn...,0
1,"Rdtanya replace broken glass, broken chargernya",0
2,Nyesel bngt dsni shopping antecedent photo mes...,0
3,Sent a light blue suit goods ga want a refund,0
4,Pendants came with dents and scratches on its ...,0
...,...,...
146806,Excellent product quality delivery speed is ve...,4
146807,thanks gan,4
146808,Awesome awesome quality merchandise value CP ...,4
146809,Nice Packing boxes made effective price .........,4


In [9]:
#mengambil sampel data dari masing masing labels agar data yang kita miliki menjadi balance
s_1 = df_train_preprocessed[df_train_preprocessed['labels']==4].sample(20000,replace = True)
s_2 = df_train_preprocessed[df_train_preprocessed['labels']==3].sample(20000,replace = True)
s_3 = df_train_preprocessed[df_train_preprocessed['labels']==2].sample(20000,replace = True)
s_4 = df_train_preprocessed[df_train_preprocessed['labels']==1].sample(20000,replace = True)
s_5 = df_train_preprocessed[df_train_preprocessed['labels']==0].sample(20000,replace = True)
df_train_preprocessed = pd.concat([s_1,s_2,s_3,s_4,s_5])

print(df_train_preprocessed.shape)
print(df_train_preprocessed['labels'].value_counts(normalize = True))

(100000, 2)
4    0.2
3    0.2
2    0.2
1    0.2
0    0.2
Name: labels, dtype: float64


In [46]:
#Function untuk Cleansing Data
import string, re

def cleansing(data):
  #lower text
  data = data.lower()

  #hapus tanda baca
  remove = string.punctuation
  translator = str.maketrans(remove, ' '*len(remove))
  data = data.translate(translator)

  #hapus ASCII dan UNICODE
  data = data.encode('ascii', 'ignore').decode('utf-8')
  data = re.sub(r'[^\x00-\x7f]','r',data)

  #remove newline
  data = data.replace('\n', ' ')

  return data

In [13]:
#Aplikasikan Function cleansing ke dataset kita
reviews = []

for index,row in df_train_preprocessed.iterrows():
  reviews.append(cleansing(row['review']))

df_train_preprocessed['review'] = reviews
df_train_preprocessed.head()

,review,labels
140939,long delivery take 22 days but were very dep ...,4
136794,fast delivery,4
107828,awesome awesome speed of the ship s delivery ...,4
128166,product quality is very good speed of deliver...,4
138711,okeee bag more gedeeee hasiilnya p...,4


# **2. Text Preprocessing**

In [14]:
#install library Sastrawi
!pip install Sastrawi

     |████████████████████████████████| 215kB 2.8MB/s 


In [15]:
#import library
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

#contoh
kalimat = 'saya mencoba dengan library sastrawi pada project kali ini'
stop = stopword.remove(kalimat)
print(stop)

mencoba library sastrawi project kali


In [16]:
#aplikasikan pada dataset kita
reviews = []

for index,row in df_train_preprocessed.iterrows():
  reviews.append(stopword.remove(row['review']))

df_train_preprocessed['review'] = reviews
df_train_preprocessed.head()

,review,labels
140939,long delivery take 22 days but were very dep ...,4
136794,fast delivery,4
107828,awesome awesome speed of the ship s delivery ...,4
128166,product quality is very good speed of deliver...,4
138711,okeee bag more gedeeee hasiilnya p...,4


In [17]:
#Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#contoh
kalimat = 'barangnya bagus banget kak, aku suka. pengirimannya juga cepet'
katadasar = stemmer.stem(kalimat)

print(katadasar)

barang bagus banget kak aku suka kirim juga cepet


In [18]:
#aplikasikan ke dataset kita

reviews = []
for index,row in df_train_preprocessed.iterrows():
  reviews.append(stemmer.stem(row['review']))

df_train_preprocessed['review'] = reviews
df_train_preprocessed.head()

,review,labels
140939,long delivery take 22 days but were very dep w...,4
136794,fast delivery,4
107828,awesome awesome speed of the ship s delivery s...,4
128166,product quality is very good speed of delivery...,4
138711,okeee bag more gedeeee hasiilnya paking rapiii...,4


In [19]:
#split training dataset menjadi 80 % training set dan 20 % test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train_preprocessed['review'], df_train_preprocessed['labels'], test_size = 0.2, stratify = df_train_preprocessed['labels'], random_state = 30)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer

corpus = [
          'Barangnya ga berkualitas',
          'Wah pengirimannya cepet banget'
]

#bow_transformer = CountVectorizer(analyzer = preprocess_data).fit(df_train_preprocessed['review'])

vectorizer = TfidfVectorizer()


#contoh
X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0.        , 0.57735027, 0.57735027, 0.        , 0.57735027,
        0.        , 0.        ],
       [0.5       , 0.        , 0.        , 0.5       , 0.        ,
        0.5       , 0.5       ]])

In [21]:
#implementasi ke dataset
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

print(X_train.shape)
print(X_test.shape)

(80000, 34582)
(20000, 34582)


In [47]:
def preprocess_data(data):
  #cleansing data
  data = cleansing(data)

  #hapus stopwords
  factory = StopWordRemoverFactory()
  stopword = factory.create_stop_word_remover()
  data = stopword.remove(data)

  #Stemming
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  data = stemmer.stem(data)

  #count vectorizer
  data = vectorizer.transform([data])

  return data

# **4. Modelling**

In [23]:
#Kali ini kita menggunakan Algoritma Naive Bayes yaitu MultinomialNB karena data kita memiliki multi class
from sklearn.naive_bayes import MultinomialNB

In [24]:
MNB = MultinomialNB()
MNB.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
from sklearn import metrics
predicted = MNB.predict(X_test)
accuracy_score_mnb = metrics.accuracy_score(predicted, y_test)

In [26]:
print('Accuracy Score MNB = ' +str('{:04.2f}'.format(accuracy_score_mnb*100))+ '%')

Accuracy Score MNB = 54.16%


# **5. Prediction on Test Dataset**

In [29]:
#Read test dataset dan ubah kolom index menjadi 'review_id'
df_test = pd.read_csv('test.csv', index_col = ['review_id'])
#Ubah kolom 'review' menjadi 'messages'
df_test = df_test.rename(columns = {'review' : 'messages'})
#tampilkan 5 data pertama
df_test.head()

,messages
review_id,
1,"Great danger, cool, motif and cantik2 jg model..."
2,One of the shades don't fit well
3,Very comfortable
4,Fast delivery. Product expiry is on Dec 2022. ...
5,it's sooooo cute! i like playing with the glit...


In [32]:
#Copy df_test ke variabel baru yaitu df_test_final
df_test_final = df_test.copy()

df_test_final

,messages
review_id,
1,"Great danger, cool, motif and cantik2 jg model..."
2,One of the shades don't fit well
3,Very comfortable
4,Fast delivery. Product expiry is on Dec 2022. ...
5,it's sooooo cute! i like playing with the glit...
...,...
60423,Product has been succesfully ordered and shipp...
60424,Opening time a little scared. Fear dalemnya de...
60425,The product quality is excellent. The origina...


In [35]:
#Masukkan seluruh isi kolom 'messages' di df_test_final ke array test_reviews
test_reviews = []
for index,row in df_test_final.iterrows():
  test_reviews.append(row['messages'])

df_test_final['messages'] = test_reviews
df_test_final.head()

,messages
review_id,
1,"Great danger, cool, motif and cantik2 jg model..."
2,One of the shades don't fit well
3,Very comfortable
4,Fast delivery. Product expiry is on Dec 2022. ...
5,it's sooooo cute! i like playing with the glit...


In [65]:
#Buat pipeline untuk melakukan prediksi terhadap test dataset
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
pipeline = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())
])

In [66]:
#HANYA PERCOBAAN UNTUK MEMASTIKAN MODEL BEKERJA VIA PIPELINE
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [67]:
#HANYA PERCOBAAN UNTUK MEMASTIKAN MODEL BEKERJA VIA PIPELINE

pred_rate = pipeline.predict(X_test)

In [68]:
#HANYA PERCOBAAN UNTUK MEMASTIKAN MODEL BEKERJA VIA PIPELINE

from sklearn.metrics import classification_report

print(classification_report(y_test,pred_rate))

              precision    recall  f1-score   support

           0       0.63      0.78      0.70      4000
           1       0.59      0.62      0.61      4000
           2       0.55      0.49      0.52      4000
           3       0.48      0.46      0.47      4000
           4       0.47      0.41      0.44      4000

    accuracy                           0.55     20000
   macro avg       0.54      0.55      0.55     20000
weighted avg       0.54      0.55      0.55     20000



In [70]:
#Ubah seluruh isi kolom 'messages' menjadi vektor agar bisa dilakukan prediksi oleh model
vector_msg = vectorizer.transform(df_test_final['messages'])

In [71]:
prediction = pipeline.predict(vector_msg)

In [72]:
#Masukkan hasil prediksi kedalam test dataset dengan nama kolom 'rating'
df_test_final['Rating'] = prediction
df_test_final

,messages,Rating
review_id,,
1,"Great danger, cool, motif and cantik2 jg model...",1
2,One of the shades don't fit well,0
3,Very comfortable,4
4,Fast delivery. Product expiry is on Dec 2022. ...,0
5,it's sooooo cute! i like playing with the glit...,1
...,...,...
60423,Product has been succesfully ordered and shipp...,3
60424,Opening time a little scared. Fear dalemnya de...,0
60425,The product quality is excellent. The origina...,3


In [73]:
#Hapus kolom 'messages' agar di test dataset yang tersisa hanya kolom 'review_id' dan 'Rating'
df_test_final = df_test_final.drop(columns = 'messages')
df_test_final.head()

,Rating
review_id,
1,1
2,0
3,4
4,0
5,1


In [74]:
#Ubah nilai label menjadi nilai semula
rating_mapper_decode = {0: 1,
                        1: 2,
                        2: 3,
                        3: 4,
                        4: 5}

df_test_final['Rating'] = df_test_final['Rating'].map(rating_mapper_decode)


In [75]:
df_test_final.head()

,Rating
review_id,
1,2
2,1
3,5
4,1
5,2


In [85]:
df_test_final.reset_index(level=0, inplace=True)
df_test_final

,index,review_id,Rating
0,0,1,2
1,1,2,1
2,2,3,5
3,3,4,1
4,4,5,2
...,...,...,...
60422,60422,60423,4
60423,60423,60424,1
60424,60424,60425,4
60425,60425,60426,2


In [86]:
df_test_final = df_test_final.drop(columns=['index'])
df_test_final.shape

(60427, 2)

In [87]:
df_test_final

,review_id,Rating
0,1,2
1,2,1
2,3,5
3,4,1
4,5,2
...,...,...
60422,60423,4
60423,60424,1
60424,60425,4
60425,60426,2


# **6.  *Tambahan (Jika ingin Menggunakan Algoritma Lain)***

In [53]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel = 'linear')

In [54]:
#lakukan prediksi dataset
clf.fit(X_train,y_train)
predict = clf.predict(X_test)

In [65]:
#import library untuk evaluasi model
from sklearn.metrics import accuracy_score

In [59]:
# f1_score
#print("f1 score hasil prediksi adalah: ")
#print(f1_score(y_test, predict))

# accuracy score
print("accuracy score hasil prediksi adalah: ")
print(accuracy_score(y_test, predict))

# precision score
#print("precision score hasil prediksi adalah: ")
#print(precision_score(y_test, predict))

# recall score
#print("recall score hasil prediksi adalah: ")
#print(recall_score(y_test, predict))

accuracy score hasil prediksi adalah: 
0.58105


In [60]:
review_positif = 'kualitasnya oke banget, emang keren deh toko ini'
review_negatif = 'responnya lambat banget, jangan belanja di toko ini deh'

In [62]:
print(clf.predict(preprocess_data(review_positif)))
print(clf.predict(preprocess_data(review_negatif)))

[4]
[2]


In [27]:
from sklearn.naive_bayes import BernoulliNB
BNB = BernoulliNB()
BNB.fit(X_train, y_train)
accuracy_score_bnb = metrics.accuracy_score(BNB.predict(X_test),y_test)
print('Accuracy Score BNB = ' +str('{:4.2f}'.format(accuracy_score_bnb*100))+'%')

Accuracy Score BNB = 48.52%


In [28]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier()
sgdc.fit(X_train, y_train)
train_score = sgdc.score(X_train, y_train)
valid_score = sgdc.score(X_test, y_test)
print(f'Train score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')


Train score: 0.63 ; Validation score: 0.53

